<h1 align=center><font size = 5>Web Scrapping </font></h1>

### Code to Extract Data from Web https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

In [5]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [9]:
from bs4 import BeautifulSoup
website_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
headers={'User Agent': 'Mozilla/5.0'}
response=requests.get(website_url, headers=headers)
soup=BeautifulSoup(response.content,'html.parser')
CANTable=soup.find_all('table',{'class':'wikitable sortable'})
CANTable=CANTable[0]

with open ('Canada_Neighborhood.txt','w') as r:
    for th in CANTable.find_all('th'):
        r.write("\"" + th.text.strip() + "\"" + ",")
        r.close
with open ('Canada_Neighborhood.txt','a') as r1:
    for row in CANTable.find_all('tr'):
        r1.write('\n')
        for cell in row.find_all('td'):
            r1.write("\"" + cell.text.strip() + "\"" + ",")
            r1.close

### Code to import the data from text file to DataFrame and Wrangling the data

In [10]:
import pandas as pd
import numpy as np
df=pd.read_csv('Canada_Neighborhood.txt', sep=",", names=["Postcode", "Borough", "Neighbourhood"],usecols=["Postcode", "Borough", "Neighbourhood"], header=0)

#striping unwanted spaces.
df.rename(columns=lambda x: x.strip())
df['Postcode'] = df['Postcode'].str.strip()
df['Borough'] = df['Borough'].str.strip()
df['Neighbourhood'] = df['Neighbourhood'].str.strip()

#deleting Not assigned value from Borough
df = df[df.Borough != 'Not assigned']

#replacing Not assigned value of Neighbourhood with Borough column value
df['Neighbourhood'] = np.where(df['Neighbourhood'] == "Not assigned", df['Borough'], df['Neighbourhood'])

#Concatenating same Neighbourhood values group by Postcode
df['Neighbourhood']= df[['Postcode','Borough','Neighbourhood']].groupby(['Postcode'])['Neighbourhood'].transform(lambda x: ', '.join(x))
df=df[['Postcode','Borough','Neighbourhood']].drop_duplicates(inplace=False)
df.reset_index(drop=True, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [11]:
df.count()

Postcode         103
Borough          103
Neighbourhood    103
dtype: int64